In [1]:
cd ..

/home/dmitriishubin/Desktop/physionet-challenge-2020


In [ ]:
#### insert the name of the last run
!tensorboard --logdir=runs/May11_01-21-42_dmitrii

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import gc
import ast
import json
import os

from metrics import Metric
from utils.KPI_plots import plot_confusion_matrix

In [3]:
def load_waveform(data_path,name):
    
    signal = np.load(data_path+name+'.npy')
        
    return signal

def load_label(data_path,name):
    
    label = json.load(open(data_path+name+'.json'))
        
    return label

In [4]:
%matplotlib qt

# Settings

In [5]:
competition_metric = Metric()

# Dataset A

In [ ]:
DEBUG_PATH = './data/CV_debug/A/'
DATA_PATH = './data/A/formatted/'

list_records = [i[:-5] for i in os.listdir(DEBUG_PATH) if i.find('.json')!=-1]


In [ ]:
# Main processing pipeline
scores_errors = np.array([])
scores_competition = np.array([])
records = np.array([]).astype(np.str)
labels = [] #np.array([])
preds = [] #np.array([])

#load all records
for record in list_records:
    
    label = load_label(DATA_PATH,record)
    records = np.append(records,label['filename'])
    label = label['labels_training_merged']
    
    pred = load_label(DEBUG_PATH,record)
    pred = pred['predicted_label']
    
    #calc score for each record
    label = np.array(label).reshape(1,-1)
    pred = np.array(pred).reshape(1,-1)
    
    scores_competition = np.append(scores_competition,competition_metric.compute(label, pred))
    scores_errors = np.append(scores_errors,np.sum(np.abs(label - pred)))
    
    #add predictions and labels for overall KPI estimation
    labels.append(label[0,:])#labels = np.append(labels,label,axis=0)
    preds.append(pred[0,:])#preds = np.append(preds,pred,axis=0)

    
preds = np.array(preds)
labels = np.array(labels)
#plot modified consustion matrix
matrix = competition_metric.compute_modified_confusion_matrix(labels, preds)
label_names = ['IAVB'
'AF',
'AFL',
'Brady',
'CRBBB',
'IRBBB',
'LAnFB',
'LAD',
'LBBB',
'LQRSV',
'NSIVCB',
'PR',
'PAC',
'PVC',
'LPR',
'LQT',
'QAb',
'RAD',
'RBBB',
'SA',
'SB',
'SNR',
'STach',
'SVPB',
'TAb',
'TInv',
'VP'
 ]


plot_confusion_matrix(matrix,label_names)

In [ ]:
print('Metric for dataset A: ',competition_metric.compute(labels, preds))

In [ ]:
#keeep records with errors only
records = records[np.where(scores_errors > 0)]
scores_errors = scores_errors[np.where(scores_errors > 0)]

#TODO: clarify the type of sort
score_order = np.argsort(scores_errors)
records = records[score_order[::1]]

#save a list of files in csv
pd.DataFrame(records).to_csv('./data/dataset_A_list.csv')

In [ ]:
def plot_record(record):
    
    ecg = np.load(DATA_PATH+record+'.npy')
    #ecg = preprocessing.run(ecg)
    meta = json.load(open(DATA_PATH+ f'{record}.json'))
    
    pred = load_label(DEBUG_PATH,record)
    pred = pred['predicted_label']
    
    #heatmap = np.array(data[record]['heatmap'],dtype=np.float)
    
    #plot the data
    fig,ax = plt.subplots(figsize=(20,20))
    fig.suptitle(record+', labels: '+str(meta['labels_full']))
    for i in range(12):
        ax.plot(ecg[:,i]+500*i)
    plt.show()
    
    print('Predictions: ',np.array(pred))
    #print('Label:       ',meta['labels_training_merged'])
    return np.array(pred)

record = records[0]

pred = plot_record(record=record)


In [ ]:
np.where(pred == np.max(pred))[0][0]

# Calculate a competition score across all datasets + joint

In [ ]:
postproc = 

In [6]:
DEBUG_PATH = './data/CV_debug/'
DATASETS = ['A','B','D','E']

for dataset in DATASETS:
    list_records = [i[:-5] for i in os.listdir(f'./data/CV_debug/{dataset}/') if i.find('.json')!=-1]


In [7]:
preds = []
labels = []

for record in list_records:
    
    if record[0] == 'A':
        data_folder = 'A'

    elif record[0] == 'Q':
        data_folder = 'B'

    elif record[0] == 'I':
        data_folder = 'C'

    elif record[0] == 'S':
        data_folder = 'D'

    elif record[0] == 'H':
        data_folder = 'E'

    elif record[0] == 'E':
        data_folder = 'F'
    
    pred_folder = f'./data/CV_debug/{data_folder}/'
    data_folder = f'./data/{data_folder}/formatted/'
    
    
    label = load_label(data_folder,record)
    label = label['labels_training_merged']
    
    pred = load_label(pred_folder,record)
    pred = pred['predicted_label']
    
    label, pred = competition_metric.find_opt_thresold(np.array(label).reshape(1,-1), np.array(pred).reshape(1,-1))
    
    break

  0%|          | 0/27 [00:00<?, ?it/s]

Finding the optimal threshold


TypeError: list indices must be integers or slices, not tuple

In [ ]:
pred